In [10]:
import pandas as pd
import numpy as np
import random as rd

def get_text_class_buffer(indexes, eng_inc, eng_exc, frn_inc, frn_exc):
  text_buffer = []
  class_buffer = []
  for i in indexes:
    text_buffer.append(eng_inc[i])
    class_buffer.append(1)
    text_buffer.append(eng_exc[i])
    class_buffer.append(0)
    text_buffer.append(frn_inc[i])
    class_buffer.append(1)
    text_buffer.append(frn_exc[i])
    class_buffer.append(0)
  return text_buffer, class_buffer

In [32]:
eng_inc=[i.strip() for i in list(open("english_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
eng_exc=[i.strip() for i in list(open("english_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
frn_inc=[i.strip() for i in list(open("french_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
frn_exc=[i.strip() for i in list(open("french_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]

if len(eng_inc) != len(eng_exc):
  raise Exception("uneven english pairs")

if len(frn_inc) != len(frn_exc):
  raise Exception("uneven french pairs")

if len(eng_inc) != len(frn_inc):
  raise Exception("amounts across french and english datasets")

num_kfolds = 3 #amount of folds to split data into. 3 is best and divides equally.
all_indexes_static = set(range(len(eng_exc)))
unused_indexes = set(range(len(eng_exc))) # keep track of unused indexes
indexes_per_fold = int(len(eng_exc)/num_kfolds)

text_buffer = []
class_buffer = []
#create data for number of folds
for fold in range(num_kfolds):
  # get indexes for current fold train
  unused_fold_indexes=set(rd.sample(unused_indexes, indexes_per_fold))
  unused_indexes-=unused_fold_indexes
  # store current fold test data
  test_indexes = all_indexes_static-unused_fold_indexes
  df=pd.DataFrame()
  df["text"], df["class"] = get_text_class_buffer(test_indexes, eng_inc, eng_exc, frn_inc, frn_exc)
  df.to_csv(f"../data_ready/k_is_{fold}/test.csv", index=False)

  # local globals for this while loop.
  df_text=[]
  df_class=[]
  sample_size = 2
  previous_size = 0
  while sample_size < indexes_per_fold:
    df=pd.DataFrame()
    curr_indexes=set(rd.sample(unused_fold_indexes, sample_size-previous_size))
    unused_fold_indexes-=curr_indexes
    
    # get for sentences for every index.
    text_buffer, class_buffer = get_text_class_buffer(curr_indexes, eng_inc, eng_exc, frn_inc, frn_exc)
    # add buffer to df
    df_text.extend(text_buffer)
    df_class.extend(class_buffer)
    df["text"]=df_text
    df["class"]=df_class

    # output to csv
    df.to_csv(f"../data_ready/k_is_{fold}/train{sample_size}.csv", index=False)
    sample_size*=2
    previous_size+=len(curr_indexes)

  # clear for next fold
  text_buffer.clear()
  class_buffer.clear()




In [ ]:
# train_text_buffer.clear()
# train_class_buffer.clear()

# four_ran_pairs=ran.sample(single_example_range, 4)
# print(four_ran_pairs)

# for i in four_ran_pairs:
#   if on_english:
#     train_text_buffer.append(eng_inc[i])
#     train_class_buffer.append(1)
#     train_text_buffer.append(eng_exc[i])
#     train_class_buffer.append(0)
#   else:
#     train_text_buffer.append(frn_inc[i])
#     train_class_buffer.append(1)
#     train_text_buffer.append(frn_exc[i])
#     train_class_buffer.append(0)

#   on_english=not on_english

# train_pairs_random_4["text"]=train_text_buffer
# train_pairs_random_4["class"]=train_class_buffer

# used_index += four_ran_pairs
# train_pairs_random_4

In [9]:
# train_pairs_random_4.to_csv("../../data_ready/few_shot/train_pairs_random_4.csv", index=False)
# train_pairs_random_8.to_csv("../../data_ready/few_shot/train_pairs_random_8.csv", index=False)
# train_unrelated_random_8.to_csv("../../data_ready/few_shot/train_unrelated_random_8.csv", index=False)
# train_unrelated_random_16.to_csv("../../data_ready/few_shot/train_unrelated_random_16.csv", index=False)
# test_set.to_csv("../../data_ready/unused_pairs_for_test_data.csv", index=False)